In [4]:
# !pip install autoawq
!pip install auto-gptq
# !pip install optimum
# !pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2


In [31]:
# https://www.youtube.com/watch?v=XpoKB3usmKc
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import transformers

# from huggingface_hub import interpreter_login
# interpreter_login()

In [10]:
MODEL_NAME="databricks/dolly-v2-3b"#"TheBloke/Llama-2-7B-AWQ"#"TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='auto', trust_remote_code=False, revision='main')
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,use_fast=True) #use_fast

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [81]:
# list(model.children())[0]
# print(child)

In [84]:
# print(model)
# child_counter=0
# for child in model.children():
#    print(" child", child_counter, "is:")
#    print(child)
#    child_counter += 1

In [15]:
model.eval()
comment="great content, thank you!"
prompt=f'''[INST]{comment}[/INST]'''

inputs = tokenizer(prompt, return_tensors='pt')
output = model.generate(input_ids=inputs['input_ids'], max_new_tokens=140)


In [16]:
print(tokenizer.batch_decode(output)[0])

[INST]great content, thank you![/INST]

[INST]I'm glad you enjoyed it![/INST]

[INST]Please let me know if you have any comments or questions about the video! I'd love to hear what you thought![/INST]

[INST]Thank you! I really appreciate your feedback![/INST]

[INST]You're welcome! I hope you enjoy the rest of the video![/INST]

[INST]I sure will![/INST]

[INST]Thank you! I'm looking forward to it![/INST]

[INST]You're welcome! Have a nice day![/INST]

[INST]You too


# check prompt engineering in together.ai

We can try different prompts to find a good one

# Start fine-tuning

In [20]:
model.train() # model in training mode (dropout modules are activated)
model.gradient_checkpointing_enable() # enable gradient check pointing
model = prepare_model_for_kbit_training(model) # enable quantized training

# Setting up LoRA
# config = LoraConfig(r=8, lora_alpha=32, target_modules=['q_proj'], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
# 29% trainable parameters
config = LoraConfig(r=8, lora_alpha=32, target_modules=["dense_h_to_4h", "dense_4h_to_h", "query_key_value", "dense", "embed_out",], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
model = get_peft_model(model, config)
# LoRA trainable parameter count
model.print_trainable_parameters()


trainable params: 10,908,480 || all params: 2,785,994,560 || trainable%: 0.39154706748601836


#### Preprocess text

In [27]:
from datasets import Dataset, load_dataset
#load dataset
data = load_dataset("shawhin/shawgpt-youtube-comments")
#https://github.com/shwhint/youtube-blog/tree/main/LLMs

def tokenize_function(examples):
   # extract text
   text = examples['example']
   #tokenize and truncate text
   tokenizer.truncation_side='left'
   tokenized_inputs = tokenizer(text, return_tensors='np', truncation=True, max_length=512)
   return tokenized_inputs

    
# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

If you have a lot of data (the loaded dataset has only 60 samples) and each sample in dataset can have different length, we need to add a padding token to the end of shorter samples to make even length for all samples. For this purpose we can use data collator

In [35]:
# -----setting pad token -----
tokenizer.pad_token = tokenizer.eos_token

# ----- data collator -----
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False) # mlm=masked language modeling

In [33]:
#Hyperparameters
lr=2e-4
batch_size=4
num_epochs=10

# define training arguments
training_args = transformers.TrainingArguments(
   output_dir='tuned_model', learning_rate=lr, 
   per_device_train_batch_size=batch_size,
   per_device_eval_batch_size=batch_size,
   num_train_epochs=num_epochs,
   weight_decay=0.01,
   logging_strategy='epoch',
   evaluation_strategy='epoch',
   save_strategy='epoch',
   load_best_model_at_end=True,
   gradient_accumulation_steps=4,
   warmup_steps=2,
   fp16=True,
   optim='paged_adamw_8bit') # optim is ingredients 3

# Run training/Fine-tuning

In [36]:
trainer=transformers.Trainer(
   model=model,
   train_dataset=tokenized_data['train'],
   eval_dataset = tokenized_data['test'],
   args = training_args,
   data_collator=data_collator)
# train model
model.config.use_cache=False # slience the warnings
trainer.train()

# renable warnings
model.config.use_cache = True

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


OutOfMemoryError: CUDA out of memory. Tried to allocate 76.00 MiB (GPU 0; 15.77 GiB total capacity; 492.35 MiB already allocated; 63.56 MiB free; 494.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# After training
I could not run the code because of memory limit. But I following instructions and here is the final expected output

#### 1st generated response by fine-tuned model
<img src='out-screenshots/comment1.jpg' width=600 height=600 />

#### 1st generated response by fine-tuned model
<img src='out-screenshots/comment2.png' width=600 height=600 />
